# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [39]:
import os
import requests
from pydantic import BaseModel, EmailStr, ValidationError, Field
from crewai_tools import tool

contiguity_crewai_token = os.getenv("CONTIGUITY_CREWAI_TOKEN")

# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [40]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

## Creating Agents

In [41]:
researcher_agent = Agent(
    role="Health Tech Event Researcher",
    goal="Identify relevant Health Tech events within the next 6 months - Todays data is 23 May 2024"
    	"Gather detailed information about these events including dates, locations, themes, and key attendees."
    	"Assess the potential benefits of AI MedReview exhibiting at these events.",
    backstory=(
        "The Health Tech Event Researcher, Alex Thompson, is a data-driven analyst with a background in health informatics. Alex has a keen eye for detail and a deep understanding of the health tech industry. Having previously worked in event management and market research, Alex brings valuable insights into identifying high-impact events. Alex’s passion for leveraging AI in healthcare drives their dedication to finding the best opportunities for AI MedReview to showcase its innovations."
    ),
    allow_delegation=True,
    verbose=True
)

In [42]:
event_planner_agent = Agent(
    role="Event Planner",
    goal="Assess the cost of attending or exhibiting at each event."
    	"Determine if the event is primarily oriented towards primary care."
    	"Identify the presence of key stakeholders such as PCN Clinical Directors, Management Leads, and ICB representatives."
    	"Collect logistical details including event address, contact information, and website.",
    backstory=(
        "Taylor Morgan, the Event Planner, is a meticulous organizer with extensive experience in event management and stakeholder engagement within the healthcare sector. Taylor has a background in healthcare administration and has previously coordinated several successful conferences and seminars. Known for their excellent networking skills and attention to detail, Taylor ensures that AI MedReview selects events that offer maximum strategic value and effective engagement with key stakeholders. Taylor’s comprehensive approach ensures that every logistical aspect is covered, allowing the team to focus on impactful participation."
    ),
    allow_delegation=True,
    verbose=True
)

In [43]:
event_promoter_agent = Agent(
    role="Event Promoter",
    goal="Craft compelling copy for event promotions."
    	"Highlight AI MedReview’s unique offerings and benefits."
    	"Ensure consistent messaging across various platforms (e.g., website, social media, newsletters).",
    backstory=(
        "Jamie Lee, the Event Promoter, is a seasoned marketing specialist with a flair for creative writing and a background in digital marketing. Jamie has worked with several startups in the health tech space, helping them build their brand presence and engage with their audience effectively. Jamie’s expertise lies in crafting narratives that resonate with both technical and non-technical audiences, making complex AI concepts accessible and exciting. Their mission is to amplify AI MedReview’s visibility and highlight its contributions to the healthcare industry."
    ),
    allow_delegation=True,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [44]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [45]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [46]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [47]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [48]:
class SendEmailToolArgs(BaseModel):
    email_address: EmailStr
    email_subject: str
    email_body: str
    content_type: str = Field(default='text', description="The content type of the email, either 'text' or 'html'")

In [53]:
class EmailModel(BaseModel):
    to: EmailStr
    from_: str
    subject: str
    body: str
    contentType: str = 'text'

    def dict(self, *args, **kwargs):
        d = super().dict(*args, **kwargs)
        d['from'] = d.pop('from_')
        return d


@tool("SendEmailTool", args_schema=SendEmailToolArgs)
def send_email_tool(email_address: EmailStr, email_subject: str, email_body: str, content_type: str = 'text') -> str:
    """
    The ability to email markdown to a specified email address, via Contiguity.
    """
    email_object = EmailModel(
        to=email_address,
        from_="CrewAI Send Email Tool",
        subject=email_subject,
        body=email_body,
        contentType=content_type
    )

    headers = {
        "Authorization": f"Token {contiguity_crewai_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(
        "https://api.contiguity.co/send/email",
        headers=headers,
        json=email_object.dict()
    )

    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Example usage of SendEmailTool
tool = SendEmailTool(
    email_address="drjanduplessis@icloud.com",
    email_subject="CrewAI Send Email Tool",
    email_body="Dear User, This is the default email body from CrewAI Send Email Tool."
)

try:
    result = tool._run("Sample text")
    print(result)
except ValidationError as e:
    print(e)
except ValueError as ve:
    print(f"ValueError: {ve}")
except requests.exceptions.RequestException as re:
    print(f"RequestException: {re}")

TypeError: tool() got an unexpected keyword argument 'args_schema'

In [51]:
send_email_tool = SendEmailTool(email_address="drjanduplesssis@icloud.com", email_subject="CrewAI Research Report", email_body="{email_body}")

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [ ]:
research_task = Task(
    description=(
        "Read the AI MedReview Product Overview in the instructions folder to get familiar with AI MedReview."
        "Conduct an online search for upcoming Health Tech events where I can network with Health Tehc leaders, or promote AI MedReview in the next 6 months. Today's date is {date_today}. Starting with a websearch of the following pages. {website1} {website2} {website3}"
    	"Gather and compile event details (dates, venues, themes, key speakers, etc.)"
    	"Evaluate the potential benefits of exhibiting at each event."
        "Email a report to {email_address}, with the subject {email_subject} and your report in the format specified as the email_body."
    ),
    expected_output=("""Dear Jan,

Research Report - Health Tech Event in the next 6 months.

1. Event Name:
   - Date: [Date]
   - Venue: [Venue]
   - Theme: [Event Theme]
   - Key Speakers: [List of Key Speakers]
   - Event Website: [URL]
   - Summary: [Brief description of the event]

2. Event Name:
   - Date: [Date]
   - Venue: [Venue]
   - Theme: [Event Theme]
   - Key Speakers: [List of Key Speakers]
   - Event Website: [URL]
   - Summary: [Brief description of the event]

---

Evaluation Summary:
- Potential Benefits: [Summary of the potential benefits of exhibiting at each event]
- Recommended Events: [List of events recommended for further evaluation]

Kind regards,
CrewAI Research Agent 
"""
    ),
    tools=[directory_read_tool, file_read_tool, search_tool, send_email_tool],
    agent=researcher_agent,
)

In [ ]:
evaluation_task = Task(
    description=(
        "The Researcher Agent reviews the gathered information and make a decision regarding the most appropriate events to attend."
    	"Decide on the most promising events for AI MedReview to attend and exhibit. The Researcher then writes a Evalusatio Meeting Summary"
    ),
    expected_output=("""Evaluation Meeting Summary

Meeting Date: [Date]

Attendees:
- Researcher: [Name]
- Event Promoter: [Name]

Reviewed Events:
1. Event Name:
   - Pros: [List of pros]
   - Cons: [List of cons]
   - Decision: [Attend / Do not attend]

2. Event Name:
   - Pros: [List of pros]
   - Cons: [List of cons]
   - Decision: [Attend / Do not attend]

---

Conclusion:
- Selected Events for AI MedReview: [List of events chosen for attendance and exhibition]"""
    ),
    tools=[directory_read_tool, file_read_tool, search_tool, send_email_tool],
    output_file="evaluation_report.md",
    agent=event_planner_agent,
)

In [ ]:
promotion_task = Task(
    description=(
        "Read AI MedReview Product Overview to ensure you are familiar with our digital product. 'AI MedReview Product Overview.txt' can be found in the instructions folder."
        "Write engaging promotional content for each selected event."
    	"Prepare social media posts, blog entries, and newsletter segments."
    	"Coordinate with the marketing team to ensure timely dissemination of promotional materials."
    ),
    expected_output=("""Promotional Content for Selected Events

1. Event Name:
   - Promotional Copy: [Engaging promotional content about AI MedReview attending the event]
   - Social Media Posts: [Drafts of social media posts]
   - Blog Entry: [Draft of blog entry]
   - Newsletter Segment: [Draft of newsletter segment]

2. Event Name:
   - Promotional Copy: [Engaging promotional content about AI MedReview attending the event]
   - Social Media Posts: [Drafts of social media posts]
   - Blog Entry: [Draft of blog entry]
   - Newsletter Segment: [Draft of newsletter segment]

---

Coordination Summary:
- Coordination with Marketing Team: [Summary of the coordination efforts and timelines for dissemination]"""
    ),
    tools=[directory_read_tool, file_read_tool, search_tool, send_email_tool],
    output_file="promotional_material.md",
    agent=event_promoter_agent,
)

In [ ]:
report_task = Task(
    description=(
        "The Researcher consolidates all findings into a detailed report."
    	"The Event Promoter includes the promotional strategies and content in the report."
    	"Submit the final report to the management team of AI MedReview."
    ),
    expected_output=("""Final Health Tech Event Report

Compiled by: [Researcher Name] and [Event Promoter Name]
Date: [Date]

1. Event Name:
   - Date: [Date]
   - Venue: [Venue]
   - Theme: [Event Theme]
   - Key Speakers: [List of Key Speakers]
   - Event Website: [URL]
   - Summary: [Brief description of the event]
   - Evaluation Summary: [Pros, Cons, and Decision]
   - Promotional Content: [Engaging promotional content, social media posts, blog entry, newsletter segment]

2. Event Name:
   - Date: [Date]
   - Venue: [Venue]
   - Theme: [Event Theme]
   - Key Speakers: [List of Key Speakers]
   - Event Website: [URL]
   - Summary: [Brief description of the event]
   - Evaluation Summary: [Pros, Cons, and Decision]
   - Promotional Content: [Engaging promotional content, social media posts, blog entry, newsletter segment]

---

Conclusion:
- Selected Events for AI MedReview: [List of events chosen for attendance and exhibition]
- Promotional Strategy Summary: [Summary of the promotional strategies and content]
- Next Steps: [Recommended next steps for AI MedReview’s participation in the selected events]"""
    ),
    tools=[directory_read_tool, file_read_tool, search_tool, send_email_tool],
    output_file="final_report.md",
    agent=event_planner_agent,
)

In [ ]:
# Define a task for sending the email
email_task = Task(
    description='Send an email with the research summary',
    expected_output='Email sent confirmation',
    agent=researcher_agent,
    input_data=inputs['email_body']
)

## Creating the Crew

In [ ]:
crew = Crew(
    agents=[researcher_agent, 
            event_planner_agent, event_promoter_agent],
    
    tasks=[research_task, evaluation_task, promotion_task, report_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
# Define your inputs
# Define your inputs
inputs = {
    "website1": "https://www.eventbrite.co.uk/d/united-kingdom--london/health-tech/",
    "website2": "https://www.meetup.com/topics/healthtech/gb/17/london/",
    "email_body": "This is a summary of the research we conducted.",
    "email_address": "jan.duplessis@nhs.net",
    "email_subject": "CrewAI Research Report"
}

# Example usage of the agent to send an email
try:
    result = agent.execute_tool("SendEmailTool", inputs['email_body'])
    print(result)
except ValidationError as e:
    print(e)
except ValueError as ve:
    print(f"ValueError: {ve}")
except requests.exceptions.RequestException as re:
    print(f"RequestException: {re}")

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)